In [ ]:
"""
Functions for generating synthetic spike data.
Major change: Adding Gibbs sampler for spike trains.
---
This code implements approximate inference methods for State-Space Analysis of
Spike Correlations (Shimazaki et al. PLoS Comp Bio 2012). It is an extension of
the existing code from repository <https://github.com/tomxsharp/ssll> (For
Matlab Code refer to <http://github.com/shimazaki/dynamic_corr>). We
acknowledge Thomas Sharp for providing the code for exact inference.
In this library are additional methods provided to perform the State-Space
Analysis approximately. This includes pseudolikelihood, TAP, and Bethe
approximations. For details see: <http://arxiv.org/abs/1607.08840>
Copyright (C) 2016
Authors of the extensions: Christian Donner (christian.donner@bccn-berlin.de)
                           Hideaki Shimazaki (shimazaki@brain.riken.jp)
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
import numpy
import random
import pdb
from multiprocessing import Pool
from functools import partial

import import_ipynb
import transforms


def generate_thetas(N, O, T, mu1=-2., sigma1=50, mu2=0., sigma2=50, alpha=12., ratio_modulated=1.):
    """ Generates dynamic thetas by a Gaussian Process.
    :param int N:
        Number of cells
    :param int O:
        Order of model
    :param int T:
        Number of time bins
    :param numpy.float mu1:
        Mean of first order parameters for the Gaussian process (Default=-2)
    :param float sigma1:
        Determines how strong first order parameters change over time (Default=50)
    :param float mu2:
        Mean of couplings for the Gaussian process (Default=0)
    :param float sigma2:
        Determines how strong couplings change over time (Default=50)
    :param float alpha:
        Scalar that determines the correlations over time of the Gaussian process is (Default=12)
    :param float ratio_modulated:
        Scalar between 0 and 1, how many of the interaction should be modulated. (Default=1)
    :return:
        Matrix with dimensions (t, d) with theta parameters generated by GP.
         'd' is the dimensionality of the model
    """
    D = transforms.compute_D(N, O)
    MU = numpy.tile(mu1, (T, D))
    MU[:,N:] = mu2
    # Create covariance matrix
    X = numpy.tile(numpy.arange(T),(T,1))
    K1 = 1./alpha*numpy.exp(-(X - X.transpose()) ** 2 / (2. * sigma1 ** 2))
    K2 = 1./alpha*numpy.exp(-(X - X.transpose()) ** 2 / (2. * sigma2 ** 2))
    # Generate Gaussian processes
    L1 = numpy.linalg.cholesky(K1 + 1e-13 * numpy.eye(T))
    L2 = numpy.linalg.cholesky(K2 + 1e-13 * numpy.eye(T))
    theta = numpy.empty([T, D])
    theta[:,:N] = mu1 + numpy.dot(L1, numpy.random.randn(T, N))
    theta[:,N:] = mu2 + numpy.dot(L2, numpy.random.randn(T, D - N))
    num_non_modulated = int(numpy.around((1. - ratio_modulated)*(D - N)))
    non_modulated_idx = random.sample(range(N,D), num_non_modulated)
    theta[:, non_modulated_idx] = 0.
    return theta


def generate_stationary_thetas(N, O, T):
    """ Generates stationary thetas.
    :param int N:
        Number of cells
    :param int O:
        Order of model
    :param int T:
        Number of time bins
    :return:
        Array (t, d) with non changing thetas. 'd' is the dimensionality of the model.
    """
    th1, th2 = -3., 0.
    D = transforms.compute_D(N, O)
    th = numpy.zeros([T,D])
    th[:,:N] = th1
    th[:,N:] = th2/N*(1 + 0.5*numpy.random.randn(T,D-N))
    idx = numpy.triu_indices(N,1)
    theta_array = numpy.zeros([N,N])
    theta_array[idx[0],idx[1]] = th[0,N:]
    theta_array[idx[1],idx[0]] = th[0,N:]
    mean_thetas = numpy.mean(theta_array, axis=0)
    theta_array -= numpy.tile(mean_thetas, [N, 1])
    th[:,N:] = theta_array[idx[0],idx[1]]
    return th


def generate_spikes(p, R, seed=None):
    """
    Draws spike patterns for each of `R' trial runs from the probability mass
    specified in `p'. `p' must have T rows, one independent probability mass for
    each timestep, and 2^C columns, where C is the number of cells (maximum of
    8) involved in the spike pattern.
    :param numpy.ndarray p:
        Probability mass of spike patterns for each timestep.
    :param int R:
        Number of spike patterns to generate for each timestep.
    :returns:
        Binary matrix with dimensions (time, runs, cells), in which a `1' in
        location (t, r, c) denotes a spike at time t in run r by cell c, as a
        numpy.ndarray
    """
    # Set metadata
    T, N = p.shape[0], numpy.int(numpy.log2(p.shape[1]))
    # Initialise random seed
    numpy.random.seed(seed)
    # Set up spike patterns
    fx = transforms.enumerate_patterns(N)
    # Set up the output array (time, trials, cells)
    T, C = p.shape[0], numpy.log2(p.shape[1])
    spikes = numpy.zeros((T, R, N))
    # Iterate over each probability
    for i in range(T):
        # Draw random values from the probability distribution
        idx = random_weighted(p[i], R)
        # Extract spike patterns for each trial
        spikes[i,:,:] = fx[idx,:]

    return spikes


def random_weighted(p, R):
    """
    Draws `R' integers from the probability mass over the integers `p'.
    :param numpy.ndarray p:
        Probability mass.
    :param int R:
        Sample size to draw from `p'.
    :returns:
        `R' random numbers drawn from distribution `p', as a numpy.ndarray.
    """
    # Take a cumulative sum of the probability mass
    cs = numpy.cumsum(p)
    # Draw uniform random numbers for each timestep or trial
    rnd = numpy.random.random(R)
    # For each random value, find the index of the first weight above it
    idx = numpy.zeros(R, dtype=numpy.int)
    for i in range(R):
        idx[i] = numpy.sum(cs < rnd[i])

    return idx


def generate_spikes_gibbs(theta, N, O, R, **kwargs):
    """Generates spike trains for the model given the thetas with
    `Gibbs-Sampling.
    :param numpy.ndarray theta:
        parameters used for sampling for each time bin
    :param int N:
        Number of cells
    :param int O:
        Order of interaction
    :param int R:
        Number of runs that are generated.
    :returns:
        Binary matrix with dimensions (time, runs, cells), in which a `1' in
        location (t, r, c) denotes a spike at time t in run r by cell c, as a
        numpy.ndarray
    """
    # Set numpy seed (should be removed at some point)
    seed = kwargs.get('seed', None)
    numpy.random.seed(seed)
    # Set pre-trials
    pre_R = kwargs.get('pre_n', 100)
    steps = kwargs.get('sample_steps', 1)
    # Get number of bins
    T = theta.shape[0]
    # Initialize array for spike data
    X = numpy.zeros([T, R*steps+pre_R, N], dtype=numpy.uint8)
    # Gets subsets
    subsets = transforms.enumerate_subsets(N, O)
    # Get number of natural parameters
    D = len(subsets)
    # Initialize subset map
    subset_map = numpy.zeros([D, N])
    # Get map of relevant patterns (d,c)
    for i in range(len(subsets)):
        subset_map[i, subsets[i]] = 1
    # Count how many cells must be active for each theta
    subset_count = numpy.sum(subset_map, axis=1)
    # Draw random numbers from uniform distribution
    rand_numbers = numpy.random.rand(T, steps*R+pre_R, N)
    # Iterate over all time bins
    for t in range(T):
        # Iterate through all Runs
        cur_theta = theta[t]
        for l in range(1, steps*R+pre_R):
            # Iterate through all cells
            for i in range(N):
                # Construct pattern from trial before and
                # from neurons that have been seen in this trial
                pattern = numpy.hstack([X[t, l, :i], X[t, l-1, i:]])
                # set x^(i,t) to "1" and compute f(X) for those
                pattern[i] = 1
                fx1 = (numpy.dot(pattern, subset_map.T) == subset_count)
                # Set x^(i,t) to "0" and compute f(X) for those
                pattern[i] = 0
                fx0 = (numpy.dot(pattern, subset_map.T) == subset_count)
                # compute p( x^(i,l) = 1 || X^(1:i-1,t),X^(i+1:N,l-1) )
                prob_spike = 0.5*(1 + numpy.tanh(0.5*(numpy.dot(cur_theta,fx1)
                                                - numpy.dot(cur_theta,fx0))))
                # if smaller than probability X^(i,l) -> 1
                X[t, l, i] = numpy.greater_equal(prob_spike,
                                                 rand_numbers[t, l, i])
    # Return spike data
    return X[:, pre_R::steps, :]


def generate_spikes_gibbs_parallel(theta, N, O, R, **kwargs):
    """Generates spike trains for the model given the thetas with
    `Gibbs-Sampling <https://en.wikipedia.org/wiki/Gibbs_sampling>`_.
    This function parallelizes samplings across bins.
    :param numpy.ndarray theta:
        parameters used for sampling for each time bin
    :param int N:
        Number of cells
    :param int O:
        Order of interaction
    :param int R:
        Number of runs that are generated.
    :returns:
        Binary matrix with dimensions (time, runs, cells), in which a `1' in
        location (t, r, c) denotes a spike at time t in run r by cell c, as a
        numpy.ndarray
    """
    # Set numpy seed (should be removed at some point)
    seed = kwargs.get('seed', None)
    numpy.random.seed(seed)
    # Set pre-trials
    pre_R = kwargs.get('pre_n', 100)
    # Sample Step
    steps = kwargs.get('sample_steps', 1)
    # Sample Step
    num_proc = kwargs.get('num_proc', 1)
    # Get number of bins
    T = theta.shape[0]
    # Initialize array for spike data
    X = numpy.zeros([T, R+pre_R, N], dtype=numpy.uint8)
    # Gets subsets
    subsets = transforms.enumerate_subsets(N, O)
    # Get number of natural parameters
    D = len(subsets)
    # Initialize subset map
    subset_map = numpy.zeros([D, N])
    # Get map of relevant patterns (d,c)
    for i in range(len(subsets)):
        subset_map[i, subsets[i]] = 1
    # Count how many cells must be active for each theta
    subset_count = numpy.sum(subset_map, axis=1)
    # Draw random numbers from uniform distribution
    rand_numbers = numpy.random.rand(T, R+pre_R, N)
    # Parallel samplings at all time bins
    pool = Pool(num_proc)
    results = pool.map(partial(gibbs_sampler, X=X, theta=theta, N=N, R=R,
        pre_R=pre_R, subset_map=subset_map, subset_count=subset_count, steps=steps), range(T))
    pool.close()

    return numpy.array(results)


def gibbs_sampler(t, X, theta, N, R, pre_R, subset_map, subset_count, steps):
    """ Samples the spike data using Gibbs sampling for time bin t.
    """
    cur_theta = theta[t]
    cur_X = numpy.zeros([R*steps+pre_R, N])
    numpy.random.seed()
    rand_numbers = numpy.random.rand(R*steps + pre_R, N)

    for l in range(1, R*steps + pre_R):
        # Iterate through all cells
        for i in range(N):
            # Construct pattern from trial before and
            # from neurons that have been seen in this trial
            pattern = numpy.hstack([cur_X[l, :i], cur_X[l-1, i:]])
            # set x^(i,t) to "1" and compute f(X) for those
            pattern[i] = 1
            fx1 = (numpy.dot(pattern, subset_map.T) == subset_count)
            # Set x^(i,t) to "0" and compute f(X) for those
            pattern[i] = 0
            fx0 = (numpy.dot(pattern, subset_map.T) == subset_count)
            # compute p( x^(i,l) = 1 || X^(1:i-1,t),X^(i+1:N,l-1) )
            prob_spike = 0.5*(1 + numpy.tanh(0.5*(numpy.dot(cur_theta,fx1)
                                                - numpy.dot(cur_theta,fx0))))
            # if smaller than probability X^(i,l) -> 1
            cur_X[l, i] = numpy.greater_equal(prob_spike, rand_numbers[l, i])

    return cur_X[pre_R::steps, :]